# Cassandra data wrangling

*   Author(s): Tomaso Stefanizzi
*   Description: This notebooks aims to solve some problems before inserting the data into the Cassandra database. Cassandra doesn't support all kinds of datatypes from sql, hence some adjustments has to be made.

## Imports

In [13]:
import pandas as pd
import numpy as np

## Import the full table

In [14]:
path = 'dataset/final.csv'
df = pd.read_csv(path, sep='$')
df.shape

(367029, 41)

the shape of the database is consistent with the previous flight table

## Data Wrangling

### Data Inspection

In [15]:
df.head()

,id_basica,id_empresa,id_tipo_linha,hr_partida_real,dt_partida_real,nr_semana_partida_real,id_aerodromo_origem,hr_chegada_real,dt_chegada_real,nr_semana_chegada_real,...,nm_aerodromo_destino,nm_municipio_destino,nm_pais_destino,nm_continente_destino,icao_airport_origem,iata_airport_origem,nm_aerodromo_origem,nm_municipio_origem,nm_pais_origem,nm_continente_origem
0,20210534,1000004,2,13:21:00,2020-01-07 00:00:00,2,588,14:31:00,2020-01-07 00:00:00,2,...,AFONSO PENA,SÃO JOSÉ DOS PINHAIS,BRASIL,AMÉRICA DO SUL,SBPA,POA,SALGADO FILHO,PORTO ALEGRE,BRASIL,AMÉRICA DO SUL
1,20210535,1000004,2,13:58:00,2020-01-03 00:00:00,1,162,18:02:00,2020-01-03 00:00:00,1,...,EDUARDO GOMES,MANAUS,BRASIL,AMÉRICA DO SUL,SBKP,VCP,VIRACOPOS,CAMPINAS,BRASIL,AMÉRICA DO SUL
2,20210536,1000004,2,22:52:00,2020-01-14 00:00:00,3,162,02:35:00,2020-01-15 00:00:00,3,...,EDUARDO GOMES,MANAUS,BRASIL,AMÉRICA DO SUL,SBKP,VCP,VIRACOPOS,CAMPINAS,BRASIL,AMÉRICA DO SUL
3,20210537,1000004,2,13:55:00,2020-01-18 00:00:00,3,162,17:37:00,2020-01-18 00:00:00,3,...,EDUARDO GOMES,MANAUS,BRASIL,AMÉRICA DO SUL,SBKP,VCP,VIRACOPOS,CAMPINAS,BRASIL,AMÉRICA DO SUL
4,20210538,1000004,2,21:57:00,2020-01-08 00:00:00,2,162,02:01:00,2020-01-09 00:00:00,2,...,EDUARDO GOMES,MANAUS,BRASIL,AMÉRICA DO SUL,SBKP,VCP,VIRACOPOS,CAMPINAS,BRASIL,AMÉRICA DO SUL


In [16]:
df.columns

Index(['id_basica', 'id_empresa', 'id_tipo_linha', 'hr_partida_real',
       'dt_partida_real', 'nr_semana_partida_real', 'id_aerodromo_origem',
       'hr_chegada_real', 'dt_chegada_real', 'nr_semana_chegada_real',
       'id_equipamento', 'ds_matricula', 'id_aerodromo_destino',
       'lt_combustivel', 'nr_assentos_ofertados', 'kg_payload', 'km_distancia',
       'nr_horas_voadas', 'nr_velocidade_media', 'sg_empresa_icao',
       'sg_empresa_iata', 'nm_empresa', 'ds_tipo_empresa', 'cd_tipo_linha',
       'ds_tipo_linha', 'ds_natureza_tipo_linha', 'ds_servico_tipo_linha',
       'sg_equipamento_icao', 'ds_modelo', 'icao_airport_destino',
       'iata_airport_destino', 'nm_aerodromo_destino', 'nm_municipio_destino',
       'nm_pais_destino', 'nm_continente_destino', 'icao_airport_origem',
       'iata_airport_origem', 'nm_aerodromo_origem', 'nm_municipio_origem',
       'nm_pais_origem', 'nm_continente_origem'],
      dtype='object')

### Fix Time format
In Cassandra, the Time format is expressed in both int or string. In this case, I will split time into two columns, one for hours and one for seconds

In [17]:
df[['hour_partida_real', 'minutes_partida_real', 'seconds']] = df['hr_partida_real'].str.split(':', expand=True)
df['hour_partida_real'] = df['hour_partida_real'].astype(int)
df['minutes_partida_real'] = df['minutes_partida_real'].astype(int)
df.drop(columns=['hr_partida_real', 'seconds'], inplace=True)

df[['hour_chegada_real', 'minutes_chegada_real','seconds']] = df['hr_chegada_real'].str.split(':', expand=True)
df['hour_chegada_real'] = df['hour_chegada_real'].astype(int)
df['minutes_chegada_real'] = df['minutes_chegada_real'].astype(int)
df.drop(columns=['hr_chegada_real', 'seconds'], inplace=True)




### Fixing datetime
For the same reasons as before, I'm splitting the datetime into day, month, year columns.

In [18]:
df.dtypes

id_basica                   int64
id_empresa                  int64
id_tipo_linha               int64
dt_partida_real            object
nr_semana_partida_real      int64
id_aerodromo_origem         int64
dt_chegada_real            object
nr_semana_chegada_real      int64
id_equipamento              int64
ds_matricula               object
id_aerodromo_destino        int64
lt_combustivel              int64
nr_assentos_ofertados       int64
kg_payload                  int64
km_distancia                int64
nr_horas_voadas           float64
nr_velocidade_media       float64
sg_empresa_icao            object
sg_empresa_iata            object
nm_empresa                 object
ds_tipo_empresa            object
cd_tipo_linha              object
ds_tipo_linha              object
ds_natureza_tipo_linha     object
ds_servico_tipo_linha      object
sg_equipamento_icao        object
ds_modelo                  object
icao_airport_destino       object
iata_airport_destino       object
nm_aerodromo_d

In [19]:
df['dt_partida_real'] = pd.to_datetime(df['dt_partida_real'])
df['dt_chegada_real'] = pd.to_datetime(df['dt_chegada_real'])

df['year_partida_real'] = df['dt_partida_real'].dt.year
df['month_partida_real'] = df['dt_partida_real'].dt.month
df['day_partida_real'] = df['dt_partida_real'].dt.day

df['year_chegada_real'] = df['dt_chegada_real'].dt.year
df['month_chegada_real'] = df['dt_chegada_real'].dt.month
df['day_chegada_real'] = df['dt_chegada_real'].dt.day

df.drop(columns=['dt_partida_real', 'dt_chegada_real'], inplace=True)

In [20]:
df.head()

,id_basica,id_empresa,id_tipo_linha,nr_semana_partida_real,id_aerodromo_origem,nr_semana_chegada_real,id_equipamento,ds_matricula,id_aerodromo_destino,lt_combustivel,...,hour_partida_real,minutes_partida_real,hour_chegada_real,minutes_chegada_real,year_partida_real,month_partida_real,day_partida_real,year_chegada_real,month_chegada_real,day_chegada_real
0,20210534,1000004,2,2,588,2,96,YBC,693,3430,...,13,21,14,31,2020,1,7,2020,1,7
1,20210535,1000004,2,1,162,1,97,YBA,438,12421,...,13,58,18,2,2020,1,3,2020,1,3
2,20210536,1000004,2,3,162,3,96,YBC,438,10656,...,22,52,2,35,2020,1,14,2020,1,15
3,20210537,1000004,2,3,162,3,97,YBA,438,12547,...,13,55,17,37,2020,1,18,2020,1,18
4,20210538,1000004,2,2,162,2,96,YBC,438,10996,...,21,57,2,1,2020,1,8,2020,1,9


seems ok

In [21]:
df.dtypes

id_basica                   int64
id_empresa                  int64
id_tipo_linha               int64
nr_semana_partida_real      int64
id_aerodromo_origem         int64
nr_semana_chegada_real      int64
id_equipamento              int64
ds_matricula               object
id_aerodromo_destino        int64
lt_combustivel              int64
nr_assentos_ofertados       int64
kg_payload                  int64
km_distancia                int64
nr_horas_voadas           float64
nr_velocidade_media       float64
sg_empresa_icao            object
sg_empresa_iata            object
nm_empresa                 object
ds_tipo_empresa            object
cd_tipo_linha              object
ds_tipo_linha              object
ds_natureza_tipo_linha     object
ds_servico_tipo_linha      object
sg_equipamento_icao        object
ds_modelo                  object
icao_airport_destino       object
iata_airport_destino       object
nm_aerodromo_destino       object
nm_municipio_destino       object
nm_pais_destin

## Save into file

In [22]:
df.to_csv('dataset/final_data/full_dataset.csv', index=False, sep=';')

In [23]:
dfs = pd.read_csv('dataset/final_data/full_dataset.csv', sep=';')
dfs.head()

,id_basica,id_empresa,id_tipo_linha,nr_semana_partida_real,id_aerodromo_origem,nr_semana_chegada_real,id_equipamento,ds_matricula,id_aerodromo_destino,lt_combustivel,...,hour_partida_real,minutes_partida_real,hour_chegada_real,minutes_chegada_real,year_partida_real,month_partida_real,day_partida_real,year_chegada_real,month_chegada_real,day_chegada_real
0,20210534,1000004,2,2,588,2,96,YBC,693,3430,...,13,21,14,31,2020,1,7,2020,1,7
1,20210535,1000004,2,1,162,1,97,YBA,438,12421,...,13,58,18,2,2020,1,3,2020,1,3
2,20210536,1000004,2,3,162,3,96,YBC,438,10656,...,22,52,2,35,2020,1,14,2020,1,15
3,20210537,1000004,2,3,162,3,97,YBA,438,12547,...,13,55,17,37,2020,1,18,2020,1,18
4,20210538,1000004,2,2,162,2,96,YBC,438,10996,...,21,57,2,1,2020,1,8,2020,1,9
